In [64]:
import numpy as np
import jax
import jax.numpy as jnp
from scipy.stats import norm, pareto
from scipy.optimize import fsolve, bisect
from numba import njit, prange

from TauchenHussey1991 import tauchen_hussey_log

from functools import partial
from dataclasses import dataclass

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm as cm

from time import time
from tqdm.auto import tqdm
from numba_progress import ProgressBar
# from status import print_parameters

In [16]:
# COLOR SETTINGS
colors = ['#2B388F', '#1B75BC', '#00A1DE', '#22A8E0', '#0AA8A1', '#076F6A']
cmap = LinearSegmentedColormap.from_list('bluegreen', colors)
# cmap

In [17]:
# Use 64 bit floats to gain precision.
jax.config.update("jax_enable_x64", True)

In [72]:
# I. PARAMETERIZATION

@dataclass(frozen=True)
class ABZ2012Parameters:
    """
    Save the calibrated parameters.
    """
    # 1. CALIBRATED PARAMETERS
    β: float = 0.96          # discount factor
    r: float = 0.04          # interest rate
    δ: float = 0.1           # capital depreciation rate
    α: float = 0.65          # technology
    γ: float = 0.3           # equity issuance cost
    ψ: float = 0.25          # capital loss after default
    θ: float = 0.072         # death rate (exogenous exit)
    ρ: float = 0.86          # shock persistence
    
    # 2. ESTIMATED PARAMETERS
    c: float = 0.55          # permanent productivity
    σ: float = 0.525         # stochastic shock variance
    φ: float = 0.001         # capital adjustment cost
    ξ: float = 0.01          # credit cost
    K0: float = 0.002        # entrant starting capital
    γe: float = 0.13         # entrant equity issuance cost
    
    # 3. OTHER PARAMETERS
    num_firms: int = 15000   # number of simulated firms
    periods: int = 500       # length of simulation periods

In [73]:
# II. STATE VARIABLE DISCRETIZATION

@dataclass
class ABZ2012Grids:
    """
    Container for grids and transition matrices.
    """
    kgrid: np.ndarray  # capital grid
    bgrid: np.ndarray  # debt grid
    zgrid: np.ndarray  # combined productivity grid
    Pz: np.ndarray     # transition matrix for productivity
    # Optionally store shapes:
    knum: int = 0
    bnum: int = 0
    znum: int = 0

    def __post_init__(self):
        # Initialize shape attributes if not provided
        if self.knum == 0: self.knum = len(self.kgrid)
        if self.bnum == 0: self.bnum = len(self.bgrid)
        if self.znum == 0: self.znum = len(self.zgrid)

In [129]:
kgrid.size

50

In [135]:
@njit(parallel=True)
def _update_one_step_numba(Vc, V, d, Ω, kgrid, bgrid, y, Pz, α, β, δ, φ, γ, r):

    knum = kgrid.size
    bnum = bgrid.size
    znum = Pz.shape[0]

    Vc_new = np.zeros_like(Vc)
    V_new = np.zeros_like(V)
    d_new = np.zeros_like(d)

    Kp_policy = np.zeros_like(Vc, dtype=np.int32)
    Bp_policy = np.zeros_like(Vc, dtype=np.int32)
    D_policy = np.zeros_like(Vc)

    # Iterate over states in parallel.
    for ki in prange(knum):
        K = kgrid[ki]
        for bri in range(bnum):
            B_R = bgrid[bri]
            for zi in range(znum):
                # Suppose default value is 0 for simplicity.
                # V_default = 0.0

                ##### REPAYMENT PROBLEM #####
                net_income = y[ki, zi] - B_R # net income

                best_value = -1e20
                best_kpi = 0
                best_bpi = 0
                best_div = 0.0

                for kpi in range(knum):
                    Kp = kgrid[kpi]
                    investment = Kp - (1.0 - δ) * K
                    adj_cost = φ * (Kp - K)**2 / K

                    for bpi in range(bnum):
                        Bp = bgrid[bpi]
                        Bp_R = Ω[bpi]

                        D = net_income + Bp - investment - adj_cost
                        flow_payoff = (1 + γ * (D < 0)) * D
                        EV = np.sum(V[kpi, bpi, :] * Pz[zi, :])

                        candidate_val = flow_payoff + β*EV

                        if candidate_val > best_value:
                            best_value = candidate_val
                            best_kpi = kpi
                            best_bpi = bpi
                            best_div = D

                Vc_new[ki, bri, zi] = best_value
                d_new[ki, bri, zi] = best_value < 0
                V_new[ki, bri, zi] = best_value * (best_value >= 0)
                Kp_policy[ki, bri, zi] = best_kpi
                Bp_policy[ki, bri, zi] = best_bpi
                D_policy[ki, bri, zi] = best_div

    return Vc_new, V_new, d_new, Kp_policy, Bp_policy, D_policy

In [155]:
class ABZ2012Model:
    """
    Organizes the Arellano et al. (2012) model, including parameters, grids, and solution methods.
    """
    def __init__(self, params: ABZ2012Parameters, grids: ABZ2012Grids):
        self.params = params
        self.grids = grids

        # Create place holders for solution objects.
        self.Vc = None   # repayment (continuation) value
        self.d = None    # default decision
        self.Kp_policy = None
        self.Bp_policy = None
        self.D_policy = None

    def solve_model(self, Vc_init=None, V_init=None, d_init=None, Ω_init=None, max_iter=500, tol=1e-8, print_iteration=False):
        # Unpack the model for readability.
        α, β, δ, φ, γ, r = (self.params.α, 
                            self.params.β, 
                            self.params.δ, 
                            self.params.φ, 
                            self.params.γ, 
                            self.params.r)
        kgrid, bgrid, zgrid, Pz = (self.grids.kgrid,
                                   self.grids.bgrid,
                                   self.grids.zgrid,
                                   self.grids.Pz)
        knum, bnum, znum = self.grids.knum, self.grids.bnum, self.grids.znum

        # Initialize Vc, V and d if not provided.
        if Vc_init is None:
            Vc_init = np.zeros((knum, bnum, znum))
        if V_init is None:
            V_init = np.zeros((knum, bnum, znum))
        if d_init is None:
            d_init = np.zeros((knum, bnum, znum))
        if Ω_init is None:
            Ω_init = bgrid * (1 + r)

        # Copy to avoid modifying external arrays.
        Vc = Vc_init.copy()
        V = V_init.copy()
        d = d_init.copy()

        # Allocate policy arrays.
        Kp_policy = np.zeros((knum, bnum, znum), dtype=np.int32)
        Bp_policy = np.zeros((knum, bnum, znum), dtype=np.int32)
        D_policy = np.zeros((knum, bnum, znum))

        # Precompute output for speed.
        y = np.zeros((knum, znum))
        for zi in range(znum):
            y[:, zi] = zgrid[zi] * (kgrid ** α)

        # Value function iteration loop.
        for it in tqdm(range(max_iter)):
            TVc, TV, Td, TKp, TBp, TD = _update_one_step_numba(Vc, V, d, Ω_init, kgrid, bgrid, y, Pz, α, β, δ, φ, γ, r)

            # Check convergence and print current status.
            error = np.max(np.abs(TVc - Vc))
            if print_iteration and ((it + 1) % 10 == 0):
                print(f'Iteration {it + 1}, error = {error:.6e}')

            # Update value and policy.
            Vc = TVc
            V = TV
            Kp_policy = TKp
            Bp_policy = TBp
            D_policy = TD

            if error < tol:
                if print_iteration:
                    print('Convergence achieved!')
                    break

        # Store final solution in the model instance.
        self.Vc = Vc
        self.d = d
        self.Kp_policy = Kp_policy
        self.Bp_policy = Bp_policy
        self.D_policy = D_policy

    def print_parameters(self):
        params = self.params
        grids = self.grids
        
        print('Parameters Information:')
        print(f'> Discount factor (β)               = {params.β}')
        print(f'> Risk-free interest rate (r)       = {params.r}')
        print(f'> Capital depreciation rate (δ)     = {params.δ}')
        print(f'> Returns to scale (α)              = {params.α}')
        print(f'> Equity issuance cost (γ)          = {params.γ}')
        print(f'> Capital loss after default (ψ)    = {params.ψ}')
        print(f'> Death rate (θ)                    = {params.θ}')
        print(f'> Credit cost (ξ)                   = {params.ξ}')
        print(f'> Capital adjustment cost (φ)       = {params.φ}')
        print(f'> Entrant starting capital (K0)     = {params.K0}')
        print(f'> Entrant equity issuance cost (γe) = {params.γe} ')
        print('Grids Information:')
        print(f'> Capital grid (K)                  = Size: {kgrid.shape}, Min.: {kgrid.min()}, Max.: {kgrid.max()}')
        print(f'> Debt grid (B)                     = Size: {bgrid.shape}, Min.: {bgrid.min()}, Max.: {bgrid.max()}')
        print(f'> Permanent productivity grid (μz)  = Size: {μzgrid.shape}, Min.: {μzgrid.min()}, Max.: {μzgrid.max()}')
        print(f'> Stochastic productivity grid (ϵ)  = Size: {egrid.shape}, Min.: {egrid.min()}, Max.: {egrid.max()}')
        print(f'> Productivity grid (z = μz * ϵ)    = Size: {zgrid.shape}, Min.: {zgrid.min()}, Max.: {zgrid.max()}')

In [156]:
# Assign parameters.
params = ABZ2012Parameters()

# 1. Capital grid
knum = 50         # number of capital grid points
kmin = 1e-15      # min. value of capital
kmax = 10.0       # max. value of capital
kgrid = np.linspace(kmin, kmax, knum)

# 2. Debt repayment grid
bnum = 50         # number of debt grid points
bmin = -2.0       # min. value of debt
bmax = 5.0        # max. value of debt
bgrid = np.linspace(bmin, bmax, bnum)

# Productivity (permanent component) grid
percentiles = np.array([0.05, 0.25, 0.5, 0.75, 0.95])
μznum = len(percentiles)
μzgrid = pareto(params.c).ppf(percentiles)

# Productivity (stochastic component) grid
enum = 2
egrid, P = tauchen_hussey_log(enum, params.ρ, params.σ, mu=-params.σ**2/2)

znum = μznum * enum
zgrid = np.array([μz * e for μz in μzgrid for e in egrid])
Pz = np.kron(np.eye(μznum), P)

grids = ABZ2012Grids(kgrid, bgrid, zgrid, Pz)

In [157]:
model = ABZ2012Model(params, grids)

In [158]:
model.solve_model(max_iter=1000, print_iteration=True)

  0%|          | 0/1000 [00:00<?, ?it/s]

Iteration 10, error = 7.257200e+02
Iteration 20, error = 4.739339e+02
Iteration 30, error = 3.149346e+02
Iteration 40, error = 2.093761e+02
Iteration 50, error = 1.392000e+02
Iteration 60, error = 9.254470e+01
Iteration 70, error = 6.152674e+01
Iteration 80, error = 4.090498e+01
Iteration 90, error = 2.719497e+01
Iteration 100, error = 1.808010e+01
Iteration 110, error = 1.202024e+01
Iteration 120, error = 7.991449e+00
Iteration 130, error = 5.312976e+00
Iteration 140, error = 3.532240e+00
Iteration 150, error = 2.348348e+00
Iteration 160, error = 1.561259e+00
Iteration 170, error = 1.037976e+00
Iteration 180, error = 6.900801e-01
Iteration 190, error = 4.587878e-01
Iteration 200, error = 3.050171e-01
Iteration 210, error = 2.027853e-01
Iteration 220, error = 1.348183e-01
Iteration 230, error = 8.963161e-02
Iteration 240, error = 5.959002e-02
Iteration 250, error = 3.961739e-02
Iteration 260, error = 2.633893e-02
Iteration 270, error = 1.751098e-02
Iteration 280, error = 1.164187e-02
I